In [1]:
import pandas as pd
import json

data = [json.loads(line) for line in 
        open('/home/left/Desktop/dataMining/set2/ask2/yelp_academic_dataset_business.json', 'r')]

In [2]:
data = [x for x in data if x['city']=='Toronto']

In [3]:
data[0]

{'business_id': 'EosRKXIGeSWFYWwpkbhNnA',
 'name': 'Xtreme Couture',
 'address': '700 Kipling Avenue Etobicoke',
 'city': 'Toronto',
 'state': 'ON',
 'postal_code': 'M8Z 5G3',
 'latitude': 43.6245394916,
 'longitude': -79.5291079302,
 'stars': 3.0,
 'review_count': 16,
 'is_open': 1,
 'attributes': {'GoodForKids': 'True',
  'ByAppointmentOnly': 'True',
  'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}",
  'BikeParking': 'False'},
 'categories': 'Martial Arts, Gyms, Fitness & Instruction, Active Life',
 'hours': {'Monday': '5:30-23:0',
  'Tuesday': '5:30-23:0',
  'Wednesday': '5:30-23:0',
  'Thursday': '5:30-23:0',
  'Friday': '5:30-23:0',
  'Saturday': '8:0-19:0',
  'Sunday': '8:0-17:0'}}

In [ ]:

with open('/home/left/Desktop/dataMining/set2/ask2/yelp_academic_dataset_review.json') as reviews_file:
    #business_id = [next(reviews_file) for x in range(1000)]
   #for y in range(100):
        #review_line = next(reviews_file)
        #business_id = json.loads(review_line)
        
    for x in range(len(data)):
        for line in reviews_file:
            #review_line = next(reviews_file)
            business_id = json.loads(line)
            
            if data[x]['business_id'] == business_id['business_id']:

                print("x :",business_id['user_id']," u :",business_id['business_id'])
        break
(business_id['business_id'])

x : mMl1zXOLGJeqZEYPJhspRw  u : EosRKXIGeSWFYWwpkbhNnA
x : z-IynR8AjB65eC6xTZP_zg  u : EosRKXIGeSWFYWwpkbhNnA
x : Lu3R-B7HZB9l5-4RXO-kTA  u : EosRKXIGeSWFYWwpkbhNnA
x : XxxEv-sX_VlTyyx1iLSk0A  u : EosRKXIGeSWFYWwpkbhNnA
x : RY9N17YrTHmajVFft87OQA  u : EosRKXIGeSWFYWwpkbhNnA
x : u3UxIl4oBJSFA2WIk5PtPQ  u : EosRKXIGeSWFYWwpkbhNnA
x : bjWPN1Rob_EJH0OBRERyAg  u : EosRKXIGeSWFYWwpkbhNnA
x : I5-cHkTGgmFP1obdsxWAng  u : EosRKXIGeSWFYWwpkbhNnA
x : R1PCyy2n0NLWr1NlTz4fmQ  u : EosRKXIGeSWFYWwpkbhNnA
x : g3Er6IsrMRGjIrqVBodBPg  u : EosRKXIGeSWFYWwpkbhNnA
x : z-zbQG6l9pwzkcbfsXAgpQ  u : EosRKXIGeSWFYWwpkbhNnA
x : NMx12qi0SLxLThk_tVxl2A  u : EosRKXIGeSWFYWwpkbhNnA
x : xZT0SXeNTI0z1Z1bQk95uw  u : EosRKXIGeSWFYWwpkbhNnA
x : WeVkkF5L39888IPPlRhNpg  u : EosRKXIGeSWFYWwpkbhNnA
x : KD2xSgMe4gMHfOV5mUHBzQ  u : EosRKXIGeSWFYWwpkbhNnA
x : ctOMJkvF_AqNwY4ly3XfIQ  u : EosRKXIGeSWFYWwpkbhNnA
x : WeVkkF5L39888IPPlRhNpg  u : EosRKXIGeSWFYWwpkbhNnA


# Βημα 2

In [39]:
import pandas as pd 
import math
table_R = pd.read_csv("pruned_data.csv")
print(table_R)

        TZQSUDDcA4ek5gBd6BzcjA  qUWqjjjfpB2-4P3He5rsKw  4.0
0       TZQSUDDcA4ek5gBd6BzcjA  6n_MDeYxU1ihB38be9TkVA  4.0
1       TZQSUDDcA4ek5gBd6BzcjA  jo4KmAqlZ7vxjHIP7IIkAw  5.0
2       TZQSUDDcA4ek5gBd6BzcjA  dsAcgF6qtZy2m6d_yWCrGQ  4.0
3       TZQSUDDcA4ek5gBd6BzcjA  siaRCT2-PkyeXUVKrywcTg  4.0
4       TZQSUDDcA4ek5gBd6BzcjA  SGP1jf6k7spXkgwBlhiUVw  5.0
...                        ...                     ...  ...
197106  3L3CI2Om_2SE3T6hxrJKPQ  j_hB9Gt3VMJAbA2JyvyFjw  4.0
197107  3L3CI2Om_2SE3T6hxrJKPQ  WKcVegcUjAld1S737a_QHw  4.0
197108  3L3CI2Om_2SE3T6hxrJKPQ  1FL3oE2mqq_EFAYPd1TWUg  5.0
197109  3L3CI2Om_2SE3T6hxrJKPQ  hlrZHM4D48XiQtXh6cRg_w  1.0
197110  3L3CI2Om_2SE3T6hxrJKPQ  _cVCzKQGt23KKW1M07Yvkw  5.0

[197111 rows x 3 columns]


In [40]:
print("Sample data should have {} rows \n".format(math.ceil(len(table_R)*0.05)))
sample_data = table_R.sample(frac=0.05)
print(len(sample_data))

Sample data should have 9856 rows 

9856


In [41]:
print("After we remove the samples R should have {}".format(orig_table_R_len - len(sample_data)))
table_R = table_R.drop(sample_data.index)
print(len(table_R))

After we remove the samples R should have 187255
187255


In [18]:
bs = []
for x in range(len(data)):
    if data[x]['review_count']>=15:
        bs.append(data[x])
print(len(bs))

7602
